In [1]:
# Importing all the required libraries

import pandas as pd
import time  
import numpy as np
import copy
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Chunking the csv to 50K rows to import it faster. 

start = time.time()

chunk_size = 50000
batch_no=1
for chunk in pd.read_csv("C:/Users/sivapr/Documents/Cops/ML Project/PC MC TPH Model/Chunking Base files/PC_MC_RD_(Wk'09-Wk'23)_Untouched.csv", chunksize=chunk_size):
    chunk.to_csv('C:/Users/sivapr/Documents/Cops/ML Project/PC MC TPH Model/Chunking Base files/PC_MC_RD_Chunk' + str(batch_no) + '.csv' , index= False)
    batch_no += 1 

print(f'The file has been broken into {batch_no-1} pieces')

The file has been broken into 16 pieces


In [3]:
# Importing all the chunks into individual dataframe.

df=pd.DataFrame()
for i in np.arange(1,batch_no):
    temp_df = pd.read_csv('C:/Users/sivapr/Documents/Cops/ML Project/PC MC TPH Model/Chunking Base files/PC_MC_RD_Chunk'+ str(i) + '.csv')
    df = pd.concat([df,temp_df]) 

df.reset_index(drop=True)

mid1 = time.time()
time_taken = ((mid1-start)/60)
print(f' The {batch_no-1} files have been imported, time taken is {round(time_taken,2)} mins')

 The 16 files have been imported, time taken is 1.25 mins


In [4]:
# Understanding the shape of the data

x = df.shape
print('The untouched dataframe shape is', x)

The untouched dataframe shape is (789464, 99)


In [5]:
# setting the rows and columns as max to review all required line items
bold="\033[1m"
pd.set_option('max_rows', None)
pd.set_option("max_columns", None)
df.columns

Index(['metric', 'selected_date', 'week', 'month', 'quarter', 'year',
       'program', 'node', 'userid', 'clientworkitemid', 'asin', 'countrycode',
       'labeltype', 'time_zone', 'beagle_workitem_workstreamname',
       'beagle_node', 'beagle_workitem_workitemstatus',
       'beagle_workitem_isdeleted', 'create_time', 'create_time2',
       'create_date2', 'target_time', 'target_date', 'destinationcountrycode',
       'sub_program', 'project', 'ingestionsourcetype', 'clusterid',
       'classify_batchid', 'classify_batchstatus', 'classify_workstream',
       'classify_assignedat', 'classify_assignedat2',
       'classify_assignedat_date2', 'classify_completedat',
       'classify_completedat2', 'classify_completedat_date2',
       'classify_lastupdatedat', 'classify_lastupdatedat2',
       'classify_lastupdatedat_date2', 'classify_userid',
       'classify_supervisor_login_name', 'classify_country', 'classify_city',
       'classification', 'classification_type', 'mc_remarks', 'audi

In [6]:
# keeping the copy of untouched dataframe in a different memory

df_copy = copy.deepcopy(df)

In [7]:
# Dropping all the unnecessary columns

cols_to_drop = ['selected_date','month','quarter','userid','clientworkitemid','countrycode','labeltype','time_zone','beagle_workitem_workitemstatus',
               'beagle_workitem_isdeleted','create_time','create_time2','create_date2','target_time',
               'target_date','sub_program','ingestionsourcetype','classify_batchstatus','classify_assignedat',
                'classify_assignedat2', 'classify_completedat','classify_completedat2','classify_lastupdatedat',
               'classify_lastupdatedat_date2','classify_country','classify_lastupdatedat2','audit_batchid', 'audit_batchstatus', 'audit_workstream',
               'audit_assignedat', 'audit_assignedat2', 'audit_assignedat_date2','audit_completedat', 'audit_completedat2',
                'audit_completedat_date2','audit_lastupdatedat', 'audit_lastupdatedat2', 'audit_lastupdatedat_date2',
                'audit_userid','audit_supervisor_login_name', 'audit_country', 'audit_city','qa_decision', 'qa_error_type',
                'qa_remarks', 'audit_classify_batchid','qa_classification', 'qa_classification_type', 'audit_sample_status',
               'sme_classify_batchid', 'sme_classify_batchstatus','sme_classify_workstream', 'sme_classify_assignedat',
                 'sme_classify_assignedat2', 'sme_classify_assignedat_date2','sme_classify_completedat', 
                'sme_classify_completedat2','sme_classify_completedat_date2', 'sme_classify_lastupdatedat',
                'sme_classify_lastupdatedat2', 'sme_classify_lastupdatedat_date2','sme_classify_userid',
                'sme_classify_supervisor_login_name','sme_classify_country', 'sme_classify_city', 'sme_classification',
                'sme_selectedstatus', 'sme_selectederror', 'sme_remarks','client_classification', 'client_remarks',
                'r1_workstream','r2_workstream','audit_classify_n_pts', 'sme_classify_n_pts', 'snapshot_day']

df.drop(columns=cols_to_drop,axis=1,inplace=True)

In [8]:
df.head()
df.destinationcountrycode.isnull().sum()

0

In [9]:
# Analysing the data shape and reviewing the dropped rows and column counts

y = df.shape
print('The updated dataframe shape is', y)
dropped_col = (x[1] - y[1])
dropped_rows = (x[0] - y[0])
print('The number of columns dropped are', dropped_col)
print('The number of rows dropped are', dropped_rows)

The updated dataframe shape is (789464, 23)
The number of columns dropped are 76
The number of rows dropped are 0


In [10]:
# Removing unnecessary string from the week column and converting it to an integer.
df['week'] = (df.week.apply(lambda x: x.replace('2021-',''))).astype(int)

In [11]:
# Reviewing the unique weeks.
display(df.week.unique())

array([17, 16, 19, 18, 14, 13, 15, 11, 12, 10,  9, 20, 21, 23, 22])

In [12]:
# Creating pivot table/crosstab to have better understanding of the data.

df_cnt_pivot = pd.crosstab(df['week'],df['node'],margins=True,margins_name='Grand Total',normalize=False)
df_cnt_pivot.reset_index(drop=False, inplace=True)
df_freq_pivot = pd.crosstab(df['week'],df['node'],margins=True,margins_name='Grand Total',normalize=True)
df_freq_pivot.reset_index(drop=False, inplace=True)
df_cnt_pivot['AMM_dist%'] = round(df_cnt_pivot['AMM']/df_cnt_pivot['Grand Total'],3)
df_cnt_pivot['BLR_dist%'] = round(df_cnt_pivot['BLR']/df_cnt_pivot['Grand Total'],3)
df_cnt_pivot['SZX_dist%'] = round(df_cnt_pivot['SZX']/df_cnt_pivot['Grand Total'],3)
df_cnt_pivot = df_cnt_pivot.reindex(columns=['week', 'AMM', 'BLR', 'SZX', 'AMM_dist%', 'BLR_dist%','SZX_dist%','Grand Total'])

In [13]:
# printing the pivot table

print(bold+'ASIN Count & Node level data'+bold)
display(df_cnt_pivot)
print(bold+'Volume Distribution data'+bold)
display(round(df_freq_pivot,3))

ASIN Count & Node level data


node,week,AMM,BLR,SZX,AMM_dist%,BLR_dist%,SZX_dist%,Grand Total
0,9,3945,40765,362,0.088,0.904,0.008,45072
1,10,9175,34322,385,0.209,0.782,0.009,43882
2,11,4785,29561,395,0.138,0.851,0.011,34741
3,12,8933,37201,398,0.192,0.799,0.009,46532
4,13,8588,45530,233,0.158,0.838,0.004,54351
5,14,22650,58310,245,0.279,0.718,0.003,81205
6,15,5006,50051,203,0.091,0.906,0.004,55260
7,16,3882,47360,322,0.075,0.918,0.006,51564
8,17,4232,56773,570,0.069,0.922,0.009,61575
9,18,5841,55824,174,0.094,0.903,0.003,61839


Volume Distribution data


node,week,AMM,BLR,SZX,Grand Total
0,9,0.005,0.052,0.000,0.057
1,10,0.012,0.043,0.000,0.056
2,11,0.006,0.037,0.001,0.044
3,12,0.011,0.047,0.001,0.059
4,13,0.011,0.058,0.000,0.069
5,14,0.029,0.074,0.000,0.103
6,15,0.006,0.063,0.000,0.070
7,16,0.005,0.060,0.000,0.065
8,17,0.005,0.072,0.001,0.078
9,18,0.007,0.071,0.000,0.078


In [14]:
## Cluster including batch_id ##

#Replacing blanks with Classified status
df.reset_index(drop=True, inplace=True)
df['classification_type'] = df['classification_type'].fillna('Classified')

#Replacing the the NA cells in cluster
count_cluster_col = df['asin'].count()
replace_value = pd.Series(np.arange(count_cluster_col))
df['clusterid'] = df['clusterid'].fillna(replace_value)

# Forming post classification cluster and counting clusters
df['clusterid'] = df['clusterid'].apply(str)
df['new_clusterid'] = (df['node']).map(str) + (df['classify_assignedat_date2']).map(str) + (df['classify_userid']).map(str) + (df['clusterid']).map(str) + (df['classify_batchid']).map(str) + (df['classification']).map(str)
df['cluster_freq'] =df.groupby(by='new_clusterid')['new_clusterid'].transform('count')

#Adding seperate columns for all the cluster size
cluster_cols_w_batch = []
cluster_max_size = 10
cluster_range= np.arange(1,cluster_max_size+1)

for i in cluster_range:
    df.loc[df['cluster_freq'] < i+1,['cluster_g' + str(i)]] = False
    lst1='cluster_g' + str(i)
    cluster_cols_w_batch.append(lst1)

df[cluster_cols_w_batch] = df[cluster_cols_w_batch].fillna(True)

In [15]:
# printing the cluster% with batch_id for reference.

print(bold+'cluster% with respect to count variation (with batch)'+bold)
for i in cluster_cols_w_batch:
    display(df[i].value_counts(normalize=True,sort=False))

cluster% with respect to count variation (with batch)


False    0.347617
True     0.652383
Name: cluster_g1, dtype: float64

False    0.392696
True     0.607304
Name: cluster_g2, dtype: float64

False    0.458094
True     0.541906
Name: cluster_g3, dtype: float64

False    0.501911
True     0.498089
Name: cluster_g4, dtype: float64

...

False    0.561998
True     0.438002
Name: cluster_g6, dtype: float64

False    0.58381
True     0.41619
Name: cluster_g7, dtype: float64

False    0.604219
True     0.395781
Name: cluster_g8, dtype: float64

False    0.622345
True     0.377655
Name: cluster_g9, dtype: float64

False    0.638407
True     0.361593
Name: cluster_g10, dtype: float64

In [16]:
## Cluster excluding batch_id ##

df['new_clusterid_wo_batch'] = (df['node']).map(str) + (df['classify_assignedat_date2']).map(str) + (df['classify_userid']).map(str) + (df['clusterid']).map(str) + (df['classification']).map(str)
df['cluster_freq_wo_batch'] =df.groupby(by='new_clusterid_wo_batch')['new_clusterid_wo_batch'].transform('count')

cluster_cols_wo_batch = []
cluster_max_size = 10
cluster_range= np.arange(1,cluster_max_size+1)

for i in cluster_range:
    df.loc[df['cluster_freq_wo_batch'] < i+1,['cluster_wo_batch_g' + str(i)]] = False
    lst2='cluster_wo_batch_g' + str(i)
    cluster_cols_wo_batch.append(lst2)

df[cluster_cols_wo_batch] = df[cluster_cols_wo_batch].fillna(True)

In [17]:
# printing the cluster% without batch_id for reference.

print(bold+'cluster% with respect to count variation (without batch)'+bold)
for i in cluster_cols_wo_batch:
    display(df[i].value_counts(normalize=True,sort=False))

cluster% with respect to count variation (without batch)


False    0.346642
True     0.653358
Name: cluster_wo_batch_g1, dtype: float64

False    0.391606
True     0.608394
Name: cluster_wo_batch_g2, dtype: float64

False    0.456986
True     0.543014
Name: cluster_wo_batch_g3, dtype: float64

False    0.500742
True     0.499258
Name: cluster_wo_batch_g4, dtype: float64

...

False    0.560891
True     0.439109
Name: cluster_wo_batch_g6, dtype: float64

False    0.582685
True     0.417315
Name: cluster_wo_batch_g7, dtype: float64

False    0.603104
True     0.396896
Name: cluster_wo_batch_g8, dtype: float64

False    0.621094
True     0.378906
Name: cluster_wo_batch_g9, dtype: float64

False    0.637066
True     0.362934
Name: cluster_wo_batch_g10, dtype: float64

In [18]:
## batch_id and PT cluster ##

df['new_batch_pt_cluster'] = (df['classify_batchid']).map(str) + (df['classification']).map(str)
df['cluster_freq_batch_pt'] =df.groupby(by='new_batch_pt_cluster')['new_batch_pt_cluster'].transform('count')

cluster_batch_pt = []
cluster_max_size = 10
cluster_range= np.arange(1,cluster_max_size+1)

for i in cluster_range:
    df.loc[df['cluster_freq_batch_pt'] < i+1,['batch_pt_cluster_g' + str(i)]] = False
    lst4='batch_pt_cluster_g' + str(i)
    cluster_batch_pt.append(lst4)

df[cluster_batch_pt] = df[cluster_batch_pt].fillna(True)

In [19]:
# printing the cluster% batch_id and PT  for reference.

print(bold+'cluster% with respect to count variation with batch_id and PT'+bold)
for i in cluster_batch_pt:
    display(df[i].value_counts(normalize=True,sort=False))

cluster% with respect to count variation with batch_id and PT


False    0.026931
True     0.973069
Name: batch_pt_cluster_g1, dtype: float64

False    0.046671
True     0.953329
Name: batch_pt_cluster_g2, dtype: float64

False    0.064858
True     0.935142
Name: batch_pt_cluster_g3, dtype: float64

False    0.080707
True     0.919293
Name: batch_pt_cluster_g4, dtype: float64

...

False    0.108786
True     0.891214
Name: batch_pt_cluster_g6, dtype: float64

False    0.121049
True     0.878951
Name: batch_pt_cluster_g7, dtype: float64

False    0.133402
True     0.866598
Name: batch_pt_cluster_g8, dtype: float64

False    0.14503
True     0.85497
Name: batch_pt_cluster_g9, dtype: float64

False    0.155797
True     0.844203
Name: batch_pt_cluster_g10, dtype: float64

In [20]:
# Reviewing the unique MPs in the dataset

df.destinationcountrycode.unique()

array(['UK', 'AE', 'US', 'SG', 'CA', 'AU', 'MX', 'BR', 'JP', 'DE', 'ES',
       'IT', 'FR'], dtype=object)

In [21]:
# Adding External Research Column using regex
df.loc[df['mc_remarks'].str.contains('http|www', flags= re.I, regex=True, na=False),['ext.research_w_amz']] = True
df.loc[df['mc_remarks'].str.contains('http|www', flags= re.I, regex=True, na=False) & ~df['mc_remarks'].str.contains('amazon.', flags= re.I, regex=True, na=False),['ext.research_wo_amz']] = True
df[['ext.research_w_amz','ext.research_wo_amz']] = df[['ext.research_w_amz','ext.research_wo_amz']].fillna(False)

In [22]:
# printing the external research% for reference.

print("\033[1m"+'external_research with amazon links'+"\033[1m")
display(df['ext.research_w_amz'].value_counts(normalize=False, sort=False))
display(df['ext.research_w_amz'].value_counts(normalize=True, sort=False))

print("\033[1m"+'external_research without amazon links'+"\033[1m")
display(df['ext.research_wo_amz'].value_counts(normalize=False, sort=False))
display(df['ext.research_wo_amz'].value_counts(normalize=True, sort=False))

external_research with amazon links


False    759346
True      30118
Name: ext.research_w_amz, dtype: int64

False    0.96185
True     0.03815
Name: ext.research_w_amz, dtype: float64

external_research without amazon links


False    779426
True      10038
Name: ext.research_wo_amz, dtype: int64

False    0.987285
True     0.012715
Name: ext.research_wo_amz, dtype: float64

In [23]:
# Reviewing the unique pts counts in the dataset

df.classify_n_pts.unique()

array([ 1.,  3.,  2.,  4., nan,  5.,  6.,  7.])

In [24]:
#Adding seperate columns for all the pts size
df['classify_n_pts'] = df['classify_n_pts'] .fillna(0)

n_pts_cols = []
pts_max_size = 7
pts_range= np.arange(1,pts_max_size+1)

for i in pts_range:
    df.loc[df['classify_n_pts'] < i+1,['classify_n_pts_g' + str(i)]] = False
    lst3='classify_n_pts_g' + str(i)
    n_pts_cols.append(lst3)

df[n_pts_cols] = df[n_pts_cols].fillna(True)

In [25]:
# printing the multi-pts% for reference.

print("\033[1m"+'multi-pts% with respect to count variation'+"\033[1m")
for i in n_pts_cols:
    display(df[i].value_counts(normalize=True,sort=False))

multi-pts% with respect to count variation


False    0.91698
True     0.08302
Name: classify_n_pts_g1, dtype: float64

False    0.977343
True     0.022657
Name: classify_n_pts_g2, dtype: float64

False    0.997538
True     0.002462
Name: classify_n_pts_g3, dtype: float64

False    0.999644
True     0.000356
Name: classify_n_pts_g4, dtype: float64

False    0.999911
True     0.000089
Name: classify_n_pts_g5, dtype: float64

False    0.999997
True     0.000003
Name: classify_n_pts_g6, dtype: float64

False    1.0
Name: classify_n_pts_g7, dtype: float64

In [26]:
# Checking of cluster columns have NaN values

print('NaN values in new_clusterid:',df.new_clusterid.isnull().sum())
print('NaN values in cluster_freq:',df.cluster_freq.isnull().sum())

NaN values in new_clusterid: 0
NaN values in cluster_freq: 0


In [27]:
print(df.mc_remarks.isnull().sum())
print(df.shape[0])
print(round((df.mc_remarks.isnull().sum())/(df.shape[0]),3))

561607
789464
0.711


In [28]:
# Reviewing the unique classification_type in the dataset

df.classification_type.unique()

array(['Classified', 'Unable To Classify', 'Need Further Assistance',
       'Invalid'], dtype=object)

In [29]:
# Adding new columns for UTC, invalid_NFA and capturing the data. 

df['unable_to_classify'] = df.classification_type.apply(lambda x: True if x == 'Unable To Classify' else False)
df['invalid_NFA'] = df.classification_type.apply(lambda x: True if (x == 'Invalid' or x == 'Need Further Assistance') else False)

In [30]:
# printing the UTC and Invalid_NFA for reference.

display(df['unable_to_classify'].value_counts(normalize=True))
display(df['invalid_NFA'].value_counts(normalize=True))

False    0.969412
True     0.030588
Name: unable_to_classify, dtype: float64

False    0.99579
True     0.00421
Name: invalid_NFA, dtype: float64

In [31]:
# Analysing the data shape.

df.shape

(789464, 70)

In [32]:
# Adding Assigned week number for AMM and other nodes.
df['classify_assignedat_date2'] = pd.to_datetime(df.classify_assignedat_date2, format='%m/%d/%Y')
df['weekstartwith-mon'] = df['classify_assignedat_date2'].dt.strftime('%W')
df['weekstartwith-sun'] = df['classify_assignedat_date2'].dt.strftime('%U')
df.loc[df['node'] != 'AMM',['assigned_week']] = df['weekstartwith-mon']
df.loc[df['node'] == 'AMM',['assigned_week']] = df['weekstartwith-sun']
del df['weekstartwith-mon'], df['weekstartwith-sun']

In [33]:
df['assigned_week'] = df['assigned_week'].map(int)
df['overflow'] = np.where(df['week'] == df['assigned_week'],True,False)

In [34]:
df['assigned_week'].astype(int).unique()

array([17, 16, 15, 19, 18, 14, 13, 12, 11, 10,  9,  8, 20, 21, 23, 22])

In [35]:
df.destinationcountrycode.unique()

array(['UK', 'AE', 'US', 'SG', 'CA', 'AU', 'MX', 'BR', 'JP', 'DE', 'ES',
       'IT', 'FR'], dtype=object)

In [36]:
# Reviewing count of overflow ASINs

print()
print("\033[1m"+'overflow asin count'+"\033[1m")
display(df['overflow'].value_counts())
print("\033[1m"+'overflow asin% count'+"\033[1m")
display(df['overflow'].value_counts(normalize=True))


overflow asin count


True     763989
False     25475
Name: overflow, dtype: int64

overflow asin% count


True     0.967731
False    0.032269
Name: overflow, dtype: float64

In [37]:
df.columns

Index(['metric', 'week', 'year', 'program', 'node', 'asin',
       'beagle_workitem_workstreamname', 'beagle_node',
       'destinationcountrycode', 'project', 'clusterid', 'classify_batchid',
       'classify_workstream', 'classify_assignedat_date2',
       'classify_completedat_date2', 'classify_userid',
       'classify_supervisor_login_name', 'classify_city', 'classification',
       'classification_type', 'mc_remarks', 'beagle_workitem_gl',
       'classify_n_pts', 'new_clusterid', 'cluster_freq', 'cluster_g1',
       'cluster_g2', 'cluster_g3', 'cluster_g4', 'cluster_g5', 'cluster_g6',
       'cluster_g7', 'cluster_g8', 'cluster_g9', 'cluster_g10',
       'new_clusterid_wo_batch', 'cluster_freq_wo_batch',
       'cluster_wo_batch_g1', 'cluster_wo_batch_g2', 'cluster_wo_batch_g3',
       'cluster_wo_batch_g4', 'cluster_wo_batch_g5', 'cluster_wo_batch_g6',
       'cluster_wo_batch_g7', 'cluster_wo_batch_g8', 'cluster_wo_batch_g9',
       'cluster_wo_batch_g10', 'new_batch_pt_cluste

In [38]:
df.loc[(df['destinationcountrycode'] == 'UK') &  df['classification'].str.contains(r'\bGeneral_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'AE') &  df['classification'].str.contains(r'\bToys_Non_Electrical_Child\b|\bToys_Electrical_Child\b|\bChildcare_Articles\b|\bLightbulbs\b|\bChildren_Products\b|\bRegcat_Temp_AE\b|\bToys_Child\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'US') &  df['classification'].str.contains(r'\bPowered_Scooters_Child\b|\bCarpets_Rugs\b|\bPest_Control_Products_Unsupported\b|\bPGA_FDA_Radiation\b|\bSleepwear_Child\b|\bCribs_Non_Full_Sized_Cradles_Infant\b|\bOTC_Drugs_Unsupported\b|\bEMC_Devices\b|\bToys_Child\b|\bHelmets_Bicycle\b|\bConsumer_Electrical_Batteries\b|\bConsumer_Primary_Batteries\b|\bChildren_Products\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'SG') &  df['classification'].str.contains(r'\bPacifiers_Infant\b|\bBathtubs_Infant\b|\bRockers_Infant\b|\bHigh_Chairs_Infant\b|\bConsumer_Electrical_Products_Unsupported\b|\bBathseats_Infant\b|\bHelmets_Motorcycle\b|\bBicycles_Child\b|\bNon_Powered_Scooters_Child\b|\bJewelry_Child\b|\bPest_Control_Devices_Unsupported\b|\bMedical_Devices_Unsupported\b|\bToys_General_Use\b|\bSports_General_Use\b|\bAuto_General_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'CA') &  df['classification'].str.contains(r'\bMattress_Crib_Infant\b|\bSunglasses_Child\b|\bRefrigerator_Doors\b|\bSwings_Infant\b|\bMouth_Contact_Products\b|\bFlexible_Film_Plastic_Bags\b|\bFurniture_Clothing_Storages\b|\bPacifiers_Infant\b|\bGlass_Doors_and_Enclosures\b|\bCarpets_Rugs\b|\bHelmets_Sports_Protective\b|\bPest_Control_Products_Unsupported\b|\bMotor_Vehicle_Charged_Devices\b|\bSlings_Infant\b|\bCushion_Pillows_Infant\b|\bBathseats_Infant\b|\bSleepwear_Child\b|\bCribs_Non_Full_Sized_Cradles_Infant\b|\bBaby_Monitors\b|\bBicycles_Child\b|\bEMC_Devices\b|\bToys_Child\b|\bSports_General_Use\b|\bHelmets_Bicycle\b|\bChildren_Products\b|\bTemporary_Unsupported\b|\bCE_General_Use\b|\bHome_General_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'AU') &  df['classification'].str.contains(r'\bSports_General_Use\b|\bSleepwear_Child\b|\bRockers_Infant\b|\bRattles_Infant\b|\bPressure_Equipment\b|\bPlants_and_Seeds_Unsupported\b|\bPest_Control_Devices_Unsupported\b|\bPPE_Cat_1\b|\bNon_Powered_Scooters_Child\b|\bMedical_Devices_Unsupported\b|\bJewelry_Child\b|\bHigh_Chairs_Infant\b|\bHelmets_Bicycle_Child\b|\bFurniture_General_Use\b|\bFootwear\b|\bFloating_Leisure\b|\bFeeding_Bottle_Nipples_Infant\b|\bDiving_Equipment\b|\bConsumer_Primary_Battery\b|\bCarriages_Strollers_Infant\b|\bBunk_Beds_Child\b|\bBicycles_Child\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'MX') &  df['classification'].str.contains(r'\bHigh_Chairs_Infant\b|\bVideogames_Unsupported\b|\bDVD_CD_VG\b|\bCE_General_Use\b|\bCar_Seats_and_Booster_Seats_Child\b|\bToys_Child_COFEPRIS\b|\bPlumbing\b|\bElectronic_Equipment\b|\bData_Processing_Equipment\b|\bCarriages_Strollers_Infant\b|\bWood_Untreated\b|\bWood_Treated\b|\bVitamins_and_Supplements_Unsupported\b|\bFood_Unsupported\b|\bChemical_Packaging\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'BR') &  df['classification'].str.contains(r'\bCE_General_Use\b|\bOther Equipment General_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'JP') &  df['classification'].str.contains(r'\bBook_Media_Unsupported\b|\bPurifiers_Water\b|\bHelmets_Motorcycle_Child\b|\bElectrical_Components\b|\bBedside_Sleepers_Infant\b|\bToys_Child_FSL\b|\bSpecified_Electrical_Appliances_Materials\b|\bNon_Specified_Electrical_Appliances_Materials\b|\bHIGH_PRESSURE_GAS_PRODUCTS\b|\bShopping_Carts\b|\bRear_Mounted_Bicycle_Seats_Child\b|\bHousehold_Plastic_Goods\b|\bFurniture_Wall_Fixed\b|\bFurniture_General_Use_HGL\b|\bElectrical_Appliances_Material_Non_PSE\b|\bCurtains_Carpets_Public_Space\b|\bBunk_Beds_Child\b|\bAuto_Safety_Products\b|\bAsbestos_Containing_Materials\b|\bMattress_Crib_Infant\b|\bTricycles_Child\b|\bCribs_Full_Sized_Infant\b|\bHelmets_Motorcycle_General_Use\b|\bBicycles_General_Use\b|\bCosmetics_Child\b|\bCarpets_Rugs\b|\bLaser_Products_General_Use\b|\bLuggage\b|\bCarriages_Strollers_Infant\b|\bSlings_Infant\b|\bBibs_Infant\b|\bCushion_Pillows_Infant\b|\bHigh_Chairs_Infant\b|\bChildcare_Articles\b|\bBicycles_Child\b|\bEMC_Devices\b|\bFurniture_General_Use\b|\bMedical_Devices_Unsupported\b|\bApparel_Child\b|\bCosmetics_General_Use\b|\bApparel_Accessories\b|\bConsumer_Electrical_Products\b|\bConsumer_Electrical_Batteries\b|\bFootwear\b|\bChildren_Products\b|\bGeneral_Use\b|\bApparel_General_Use\b|\bChemicals_Unsupported\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'DE') &  df['classification'].str.contains(r'\bGeneral_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'ES') &  df['classification'].str.contains(r'\bGeneral_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'IT') &  df['classification'].str.contains(r'\bGeneral_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df.loc[(df['destinationcountrycode'] == 'FR') &  df['classification'].str.contains(r'\bGeneral_Use\b', flags= re.I, regex=True, na=False),['complexity_h']] = True
df[['complexity_h']] = df[['complexity_h']].fillna(False)

In [39]:
df.complexity_h.value_counts(normalize=True)

False    0.774551
True     0.225449
Name: complexity_h, dtype: float64

## Fact-level Dataframe

In [40]:
# Overflow of ASINs in pivot table.

df_overflow_pivot = pd.crosstab(df['week'],df['overflow'],margins=True,margins_name='Grand Total',normalize=False)
df_overflow_pivot.reset_index(drop=False, inplace=True)
df_overflow_pivot['False%'] = round(df_overflow_pivot[False]/df_cnt_pivot['Grand Total'],3)
df_overflow_pivot['True%'] = round(df_overflow_pivot[True]/df_cnt_pivot['Grand Total'],3)
df_overflow_pivot = df_overflow_pivot.reindex(columns=['week', False, True, 'False%', 'True%','Grand Total'])

In [41]:
# Printing overflow data for reference

print("\n\033[1m"+'Overflow of ASINs at week-level'+"\033[1m")
display(df_overflow_pivot)


Overflow of ASINs at week-level


overflow,week,False,True,False%,True%,Grand Total
0,9,2265,42807,0.050,0.950,45072
1,10,1734,42148,0.040,0.960,43882
2,11,420,34321,0.012,0.988,34741
3,12,1680,44852,0.036,0.964,46532
4,13,1822,52529,0.034,0.966,54351
5,14,987,80218,0.012,0.988,81205
6,15,2387,52873,0.043,0.957,55260
7,16,1381,50183,0.027,0.973,51564
8,17,1475,60100,0.024,0.976,61575
9,18,3926,57913,0.063,0.937,61839


In [42]:
# Reshaping MP values

df['asin_count'] = 1
mp_in_df=df.destinationcountrycode.unique()
df_mp_pivot = pd.pivot_table(data=df,index=['node','week','classify_userid'],columns=['destinationcountrycode'],values='asin_count', aggfunc='mean').fillna(0).reset_index()
df_mp_pivot['no_of_mp'] = (df_mp_pivot['AE'] + df_mp_pivot['AU'] + df_mp_pivot['BR'] + df_mp_pivot['CA'] + df_mp_pivot['DE'] + df_mp_pivot['ES'] + df_mp_pivot['FR'] + df_mp_pivot['IT'] + df_mp_pivot['JP'] + df_mp_pivot['MX'] + df_mp_pivot['SG'] + df_mp_pivot['UK'] + df_mp_pivot['US']).astype(int)

for mkp in mp_in_df:
    df_mp_pivot[mkp] = [mkp if x == 1 else 'NaN' for x in df_mp_pivot[mkp]] 

df_mp_pivot['mp'] = df_mp_pivot['AE'] + ',' + df_mp_pivot['AU'] + ',' + df_mp_pivot['BR'] + ',' + df_mp_pivot['CA'] + ',' + df_mp_pivot['DE'] + ',' + df_mp_pivot['ES'] + ',' + df_mp_pivot['FR'] + ',' + df_mp_pivot['IT'] + ',' + df_mp_pivot['JP'] + ',' + df_mp_pivot['MX'] + ',' + df_mp_pivot['SG'] + ',' + df_mp_pivot['UK'] + ',' + df_mp_pivot['US']
df_mp_pivot['mp'] = (df_mp_pivot.mp.apply(lambda x: x.replace(',NaN','').replace('NaN,','')))
df_mp_pivot.drop(labels=mp_in_df,axis=1,inplace=True)

df_mp_pivot.head(10)

destinationcountrycode,node,week,classify_userid,no_of_mp,mp
0,AMM,9,ahzaidan,2,"AE,UK"
1,AMM,9,gammohn,2,"AE,UK"
2,AMM,10,fmalabe,2,"AE,UK"
3,AMM,10,gammohn,2,"AE,UK"
4,AMM,11,fmalabe,2,"AE,UK"
5,AMM,11,gammohn,2,"AE,UK"
6,AMM,12,ahzaidan,2,"AE,UK"
7,AMM,12,fmalabe,2,"AE,UK"
8,AMM,12,gammohn,2,"AE,UK"
9,AMM,13,ahzaidan,2,"AE,UK"


In [43]:
df_mp_pivot.no_of_mp.unique()

array([2, 1, 9, 8, 5, 4])

In [44]:
print(bold+'Freq Distribution of MPs'+bold)
display(df_mp_pivot.mp.value_counts())
display(df_mp_pivot.mp.value_counts(normalize=True))

Freq Distribution of MPs


UK                            318
CA                             51
AE                             50
SG                             36
AU                             30
AE,UK                          24
US                             24
JP                             23
MX                             18
BR                             12
DE,ES,FR,IT                     2
CA,DE,ES,FR,IT,MX,SG,UK,US      1
DE,ES,FR,IT,UK                  1
CA,DE,ES,FR,IT,MX,UK,US         1
ES                              1
Name: mp, dtype: int64

UK                            0.537162
CA                            0.086149
AE                            0.084459
SG                            0.060811
AU                            0.050676
AE,UK                         0.040541
US                            0.040541
JP                            0.038851
MX                            0.030405
BR                            0.020270
DE,ES,FR,IT                   0.003378
CA,DE,ES,FR,IT,MX,SG,UK,US    0.001689
DE,ES,FR,IT,UK                0.001689
CA,DE,ES,FR,IT,MX,UK,US       0.001689
ES                            0.001689
Name: mp, dtype: float64

In [45]:
df_mp_pivot.mp.unique()

array(['AE,UK', 'AE', 'UK', 'SG', 'CA', 'AU', 'US', 'MX', 'BR',
       'CA,DE,ES,FR,IT,MX,SG,UK,US', 'CA,DE,ES,FR,IT,MX,UK,US',
       'DE,ES,FR,IT,UK', 'ES', 'DE,ES,FR,IT', 'JP'], dtype=object)

In [46]:
df_mp_pivot.shape

(592, 5)

In [47]:
# Pivoting the final dataset using groupby for EDA.

df_eda_table = df.groupby(by=['node','week','classify_userid'],sort=False)[['asin_count','ext.research_w_amz', 
        'ext.research_wo_amz','cluster_g1',
       'cluster_g2', 'cluster_g3', 'cluster_g4', 'cluster_g5', 'cluster_g6',
       'cluster_g7', 'cluster_g8', 'cluster_g9', 'cluster_g10','unable_to_classify','invalid_NFA',
        'cluster_wo_batch_g1', 'cluster_wo_batch_g2', 'cluster_wo_batch_g3',
       'cluster_wo_batch_g4', 'cluster_wo_batch_g5', 'cluster_wo_batch_g6',
       'cluster_wo_batch_g7', 'cluster_wo_batch_g8', 'cluster_wo_batch_g9',
       'cluster_wo_batch_g10','batch_pt_cluster_g1', 'batch_pt_cluster_g2', 'batch_pt_cluster_g3',
       'batch_pt_cluster_g4', 'batch_pt_cluster_g5', 'batch_pt_cluster_g6',
       'batch_pt_cluster_g7', 'batch_pt_cluster_g8', 'batch_pt_cluster_g9',
       'batch_pt_cluster_g10','classify_n_pts_g1','classify_n_pts_g2', 'classify_n_pts_g3', 'classify_n_pts_g4',
       'classify_n_pts_g5','classify_n_pts_g6', 'classify_n_pts_g7','complexity_h']].sum().reset_index()

df_eda_table.head()

,node,week,classify_userid,asin_count,ext.research_w_amz,ext.research_wo_amz,cluster_g1,cluster_g2,cluster_g3,cluster_g4,cluster_g5,cluster_g6,cluster_g7,cluster_g8,cluster_g9,cluster_g10,unable_to_classify,invalid_NFA,cluster_wo_batch_g1,cluster_wo_batch_g2,cluster_wo_batch_g3,cluster_wo_batch_g4,cluster_wo_batch_g5,cluster_wo_batch_g6,cluster_wo_batch_g7,cluster_wo_batch_g8,cluster_wo_batch_g9,cluster_wo_batch_g10,batch_pt_cluster_g1,batch_pt_cluster_g2,batch_pt_cluster_g3,batch_pt_cluster_g4,batch_pt_cluster_g5,batch_pt_cluster_g6,batch_pt_cluster_g7,batch_pt_cluster_g8,batch_pt_cluster_g9,batch_pt_cluster_g10,classify_n_pts_g1,classify_n_pts_g2,classify_n_pts_g3,classify_n_pts_g4,classify_n_pts_g5,classify_n_pts_g6,classify_n_pts_g7,complexity_h
0,AMM,17,fmalabe,1983,98,66,1516,1404,1305,1233,1183,1135,1079,1015,988,958,8,0,1516,1404,1305,1233,1183,1135,1079,1015,988,958,1860,1792,1702,1646,1626,1578,1522,1474,1429,1419,124,18,2,0,0,0,0,795
1,AMM,17,ahzaidan,2249,0,0,2052,1974,1854,1774,1714,1678,1643,1611,1566,1516,0,0,2052,1974,1860,1780,1720,1684,1656,1632,1587,1527,2146,2092,2047,2007,1982,1952,1917,1909,1900,1900,67,14,0,0,0,0,0,1055
2,BLR,17,dshemant,1457,123,2,1397,1365,1335,1311,1291,1267,1260,964,955,935,30,0,1397,1365,1335,1311,1291,1267,1260,964,955,935,1443,1433,1430,1414,1414,1408,1408,1392,1392,1392,147,3,0,0,0,0,0,321
3,BLR,17,patamare,1225,261,78,777,727,676,640,630,588,574,558,540,530,103,29,777,727,676,640,630,588,574,558,540,530,1138,1074,1008,964,934,910,896,872,854,844,279,157,52,7,0,0,0,500
4,BLR,17,upooj,1995,16,16,1942,1882,1702,1538,1513,1495,1110,1102,1093,1083,18,0,1942,1882,1702,1538,1513,1495,1110,1102,1093,1083,1978,1962,1947,1939,1924,1924,1896,1888,1861,1861,46,0,0,0,0,0,0,395


In [48]:
df_eda_table = pd.merge(left=df_eda_table, right=df_mp_pivot,how='inner')

column_names = ['node', 'week', 'classify_userid', 'mp', 'no_of_mp', 'asin_count', 'ext.research_w_amz',
       'ext.research_wo_amz', 'cluster_g1', 'cluster_g2', 'cluster_g3',
       'cluster_g4', 'cluster_g5', 'cluster_g6', 'cluster_g7', 'cluster_g8',
       'cluster_g9', 'cluster_g10', 'unable_to_classify', 'invalid_NFA',
       'cluster_wo_batch_g1', 'cluster_wo_batch_g2', 'cluster_wo_batch_g3',
       'cluster_wo_batch_g4', 'cluster_wo_batch_g5', 'cluster_wo_batch_g6',
       'cluster_wo_batch_g7', 'cluster_wo_batch_g8', 'cluster_wo_batch_g9',
       'cluster_wo_batch_g10','batch_pt_cluster_g1', 'batch_pt_cluster_g2', 'batch_pt_cluster_g3',
       'batch_pt_cluster_g4', 'batch_pt_cluster_g5', 'batch_pt_cluster_g6',
       'batch_pt_cluster_g7', 'batch_pt_cluster_g8', 'batch_pt_cluster_g9',
       'batch_pt_cluster_g10','classify_n_pts_g1', 'classify_n_pts_g2', 'classify_n_pts_g3',
       'classify_n_pts_g4', 'classify_n_pts_g5', 'classify_n_pts_g6',
       'classify_n_pts_g7','complexity_h']

df_eda_table = df_eda_table.reindex(columns=column_names)

for i in df_eda_table.columns[6:]:
    df_eda_table[i+'%'] = (df_eda_table[i]/df_eda_table['asin_count'])

In [49]:
display(df_eda_table.node.value_counts())
display(round(df_eda_table.node.value_counts(normalize=True),3))

BLR    510
AMM     59
SZX     23
Name: node, dtype: int64

BLR    0.861
AMM    0.100
SZX    0.039
Name: node, dtype: float64

In [50]:
display(df_eda_table.columns)
display(df_eda_table.no_of_mp.unique())
display(df_eda_table.mp.unique())

Index(['node', 'week', 'classify_userid', 'mp', 'no_of_mp', 'asin_count',
       'ext.research_w_amz', 'ext.research_wo_amz', 'cluster_g1', 'cluster_g2',
       'cluster_g3', 'cluster_g4', 'cluster_g5', 'cluster_g6', 'cluster_g7',
       'cluster_g8', 'cluster_g9', 'cluster_g10', 'unable_to_classify',
       'invalid_NFA', 'cluster_wo_batch_g1', 'cluster_wo_batch_g2',
       'cluster_wo_batch_g3', 'cluster_wo_batch_g4', 'cluster_wo_batch_g5',
       'cluster_wo_batch_g6', 'cluster_wo_batch_g7', 'cluster_wo_batch_g8',
       'cluster_wo_batch_g9', 'cluster_wo_batch_g10', 'batch_pt_cluster_g1',
       'batch_pt_cluster_g2', 'batch_pt_cluster_g3', 'batch_pt_cluster_g4',
       'batch_pt_cluster_g5', 'batch_pt_cluster_g6', 'batch_pt_cluster_g7',
       'batch_pt_cluster_g8', 'batch_pt_cluster_g9', 'batch_pt_cluster_g10',
       'classify_n_pts_g1', 'classify_n_pts_g2', 'classify_n_pts_g3',
       'classify_n_pts_g4', 'classify_n_pts_g5', 'classify_n_pts_g6',
       'classify_n_pts_g7', 'co

array([2, 1, 8, 9, 5, 4])

array(['AE,UK', 'UK', 'US', 'SG', 'CA', 'AU', 'AE', 'MX', 'BR', 'JP',
       'CA,DE,ES,FR,IT,MX,UK,US', 'CA,DE,ES,FR,IT,MX,SG,UK,US',
       'DE,ES,FR,IT,UK', 'ES', 'DE,ES,FR,IT'], dtype=object)

In [51]:
# Captures the time taken for the code execution

mid2 = time.time()
overall_time_taken = ((mid2-start)/60)
print(f'Time taken till this is {round(overall_time_taken,2)} mins')

Time taken till this is 2.97 mins


In [52]:
# Exporting the file for EDA
# export_start = time.time()

#df_eda_table.to_csv('C:/Users/sivapr/Documents/Cops/ML Project/PC MC TPH Model/Chunking Base files/df_eda_table.csv',index=False)
df.to_csv("C:/Users/sivapr/Documents/Cops/ML Project/PC MC TPH Model/Chunking Base files/PC_MC_RD_(Wk'09-Wk'23)_Processed.csv",index=False)

# export_end = time.time()
# export_time_taken = ((mid2-start)/60)
# print(f'Time taken to export the file is {round(export_time_taken,2)} mins')

In [53]:
end = time.time()
overall_time_taken = ((end-start)/60)
print(f'Overall time taken is {round(overall_time_taken,2)} mins')

Overall time taken is 3.69 mins
